In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("training.csv")

In [3]:
df.values.shape

(7049, 31)

In [4]:
type(df.values)

numpy.ndarray

In [6]:
df.columns

Index(['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y', 'Image'],
      dtype='object')

In [7]:
df.Image.shape

(7049,)

In [8]:
type(df.Image)

pandas.core.series.Series

In [3]:
images = df.Image.values

In [4]:
import numpy as np

In [5]:
im = np.fromstring(df.Image.values[0], sep = " ")

In [6]:
images = np.zeros((df.values.shape[0], 96, 96))

In [7]:
for i in range(df.values.shape[0]):
        images[i] = np.fromstring(df.Image.values[i], sep = " ").reshape(96, 96)

In [21]:
images.shape

(7049, 96, 96)

In [8]:
coords = df.values[:, :30].astype("float")

In [25]:
coords.shape

(7049, 30)

In [9]:
isNanMatrix = np.isnan(coords)

In [10]:
rowsWithNan = isNanMatrix.any(axis=1)

In [11]:
coordsNew = coords[~rowsWithNan]
imagesNew = images[~rowsWithNan]

In [12]:
coordsNew.shape

(2140, 30)

In [34]:
imagesNew.shape

(2140, 96, 96)

In [12]:
coordsNewRounded = np.round(coordsNew).astype(int) - 1

In [13]:
coordsNewReshaped = coordsNewRounded.reshape(imagesNew.shape[0], 15, 2)

In [14]:
pixelTensor = np.zeros((imagesNew.shape[0], 96, 96, 15))
for image in range(imagesNew.shape[0]):
    for feature in range(15):
        pixelTensor[image, coordsNewReshaped[image, feature, 1], coordsNewReshaped[image, feature, 1], feature] = 1.0

In [15]:
np.unravel_index(pixelTensor[0, :, :, 0].argmax(), (96, 96))

(38, 38)

In [16]:
pixelTensor[0, 38, 65, 0]

0.0

In [49]:
coordsNewReshaped[0][0]

array([65, 38])

In [50]:
imagesNew.shape

(2140, 96, 96)

In [51]:
pixelTensor.shape

(2140, 96, 96, 15)

In [17]:
imagesNew = imagesNew.reshape(imagesNew.shape + (1,))/255

In [53]:
imagesNew[0][0][0][0]

0.9333333333333333

In [18]:
images_train = imagesNew[:1500, :, :, :]
images_test = imagesNew[1500:, :, :, :]
pixel_train = pixelTensor[:1500, :, :, :]
pizel_test = pixelTensor[1500:, :, :, :]

In [1]:
import keras

Using TensorFlow backend.


In [2]:
def softMaxAxis(x):
    return keras.activations.softmax(x, axis=1)

In [3]:
from keras.layers import Conv2D, Dropout

In [4]:
model = keras.Sequential()
model.add(Conv2D(16, (3, 3), padding="same", activation='relu', input_shape=(96, 96, 1)))
model.add(Dropout(0.2))
model.add(Conv2D(24, (3, 3), padding="same", activation='relu', input_shape=(96, 96, 1)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(96, 96, 1)))
model.add(Dropout(0.2))
model.add(Conv2D(40, (3, 3), padding="same", activation='relu', input_shape=(96, 96, 1)))
model.add(Dropout(0.2))
model.add(Conv2D(15, (3, 3), padding="same", input_shape=(96, 96, 1), activation=softMaxAxis))
#model.add(keras.activations.softmax(axis = -1))

In [5]:
model.load_weights("face_weights")

In [6]:
import cv2

In [7]:
def predict(frame):
    frame = frame.reshape(1, 96, 96, 1)
    points = model.predict(frame)[0]
    points_coords = np.unravel_index(np.argmax(points.reshape(-1, 15), axis= -1), (96, 96))
    point_coords = np.array(points_coords).T
    

In [26]:
x = np.random.rand(96, 96, 1)
predict(x)

In [8]:
face_detector = cv2.CascadeClassifier('C:\\Users\\hunte\\Anaconda3\\envs\\env2\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')

In [9]:
dev = cv2.VideoCapture(0)
try:
    while(True):
        ret, frame = dev.read()
        if (not ret):
            continue
        xSize, ySize, _ = frame.shape
        cv2.imshow("webcam", frame)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(frame_gray)
        if (len(faces)>0):
            (x,y,w,h) = faces[0]
            cv2.rectangle(frame, (x,y), (x+w, y+h), [0,0,255], 10)
            face = frame_gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (96, 96))
            coords = predict(face_resized)
            for i in range(15):
                face_resized = cv2.circle(face_resized, 
                                          (coords[i][0],coords[i][1]) 
                                           , 3, (255,0,0))
            cv2.imshow("new face", face_resized)
        
            
        cv2.imshow("faces", frame)
        if (cv2.waitKey(1) == ord('q')):
            break
except Exception as e:
    print(e)

dev.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [64]:
model.compile(optimizer=keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy)

In [66]:
model.fit(images_train, pixel_train, batch_size=32, epochs=10)

Epoch 1/10


KeyboardInterrupt: 

In [67]:
def softMaxAxis(x):
    return keras.activations.softmax(x, axis=1)

In [68]:
model = keras.models.load_model("model_face")

ValueError: Unknown activation function:softMaxAxis